<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/TwADR-L/Code/twadr_pubmed_tweet_overlap_30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
if os.path.exists('medical-concept-normalization'):
  print('folder exist')
else:
  os.mkdir(base_dir)

In [0]:
if os.path.exists('medical-concept-normalization/data_collection'):
  print('folder exist')
else:
  os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [7]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [8]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# datasets

train_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/TwADR-L/Experiment%20(Change%20test%20dataset)/Training%20set/tward_training_folder_0.csv'
valid_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/TwADR-L/Experiment%20(Change%20test%20dataset)/Validation%20set/tward_validation_folder_0.csv'
test_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/TwADR-L/Experiment%20(Change%20test%20dataset)/Testing%20set/tward_testing_overlap_30.csv'
fine_tuning_data = 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv'

In [12]:
# training data set
train = pd.read_csv(train_csv, header=None)
print(train.head())

# validating data set
valid = pd.read_csv(valid_csv,header=None)
# valid.head()

# testing data set
test = pd.read_csv(test_csv,header=None)
# test.head()

      0                      1
0  1945    Adrenergic syndrome
1  1586  Hearing Loss, Partial
2   580          Typhoid Fever
3   784      Carcinoma of lung
4   660              Pregnancy


In [14]:
# Creating DataBunch for language modelling
data_lm = TextLMDataBunch.from_csv(path, fine_tuning_data)

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [17]:
data_clas.add_test(test,label=0)

In [0]:
data_lm.save()
data_clas.save()

In [19]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'was',
 'diclofenac',
 'were',
 'on',
 'is',
 'rt',
 'sodium',
 'by',
 'that',
 'as',
 'at',
 'patients',
 'or',
 'this',
 'from',
 'drug',
 'you',
 'i',
 'mg',
 'group',
 'be',
 'disease',
 'study',
 'are',
 'treatment',
 'pain',
 'after',
 'an',
 'not',
 'it',
 'amp',
 'have',
 'p',
 'release',
 'gluten',
 'we',
 'all',
 'than',
 'free',
 'effect',
 'no',
 'new',
 'compared',
 'more',
 'using',
 'groups',
 'your',
 'both',
 'effects',
 'our',
 'h',
 'two',
 'time',
 'its',
 'about',
 'significantly',
 'between',
 'inflammatory',
 'significant',
 'my',
 'drugs',
 'these',
 'use',
 'out',
 'but',
 'can',
 'anti',
 'day',
 'used',
 'which',
 'voltaren',
 'dose',
 'potassium',
 'has',
 'results',
 'also',
 'placebo',
 'do',
 'glutenfree',
 'what',
 's',
 'there',
 'efficacy',
 'one',
 'may',
 'celiac',
 'nt',
 'm',
 'ds',
 'treated',
 'day

In [20]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [21]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.197753,4.623670,0.251851,23:00


In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.436646,4.280147,0.285658,23:01


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.109447,4.239304,0.290086,23:06


In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.057153,4.207888,0.293545,23:09


In [0]:
learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('tuning_pubmed_bigtweet_overlap_0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.096726,6.286728,0.104348,00:56


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.232969,4.998733,0.139130,00:57


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.746493,4.706879,0.200000,01:12


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.544164,4.552039,0.226087,01:13
1,5.343126,4.478106,0.226087,01:08


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.128808,4.328499,0.217391,01:06
1,4.951905,4.310407,0.200000,01:06


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.820769,4.229099,0.243478,01:08
1,4.677196,4.242352,0.252174,01:11


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.530648,4.155343,0.234783,01:08
1,4.383674,4.125904,0.243478,01:12


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.215895,4.036333,0.260870,01:11
1,4.120699,4.076908,0.208696,01:06


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.961344,4.076978,0.286957,01:08
1,3.909445,3.956740,0.286957,01:11
2,3.714388,3.976409,0.252174,01:13
3,3.571216,3.971912,0.243478,01:11


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.481093,3.914340,0.243478,01:09
1,3.457930,3.966869,0.252174,01:08
2,3.306016,3.923901,0.243478,01:06
3,3.182087,3.944561,0.260870,01:05


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.076191,3.954099,0.243478,01:12
1,3.013112,4.028509,0.234783,01:07
2,2.841556,3.940191,0.278261,01:14
3,2.772004,4.008952,0.260870,01:08


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.652328,4.038305,0.260870,01:13
1,2.752751,4.092868,0.226087,01:06
2,2.607561,4.023578,0.278261,01:09
3,2.462204,4.003863,0.286957,01:07


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.376794,4.048458,0.252174,01:06
1,2.380484,4.082301,0.234783,01:09
2,2.273695,4.024547,0.252174,01:12
3,2.207670,4.079610,0.243478,01:08


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.170852,4.154757,0.278261,01:08
1,2.201084,4.208361,0.260870,01:13
2,2.147114,4.148467,0.278261,01:08
3,2.070100,4.145962,0.269565,01:07


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.858366,4.146889,0.252174,01:14
1,1.951072,4.316057,0.260870,01:09
2,1.921545,4.355181,0.260870,01:13
3,1.913205,4.401814,0.260870,01:06


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.874783,4.398427,0.278261,01:09
1,1.866931,4.379682,0.278261,01:06


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.850635,4.416687,0.278261,01:11
1,1.786559,4.378592,0.252174,01:08


In [44]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.818326,4.438716,0.286957,01:10


In [0]:
# save the best model

learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_last')

# Part three: Predict on the test dataset

In [46]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [47]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

205
205
[2099, 2099, 1308, 1247, 2088, 1639, 744, 803, 135, 1411, 1049, 744, 854, 1639, 860, 87, 854, 711, 965, 1072, 106, 803, 931, 744, 1802, 171, 1077, 744, 1307, 2099, 1935, 1639, 1308, 744, 12, 309, 744, 744, 244, 1411, 2099, 1407, 1932, 171, 106, 1935, 1307, 621, 2097, 854, 1811, 574, 744, 194, 135, 1546, 1583, 2182, 2088, 1077, 92, 375, 1932, 1376, 744, 931, 744, 1935, 1956, 860, 2099, 1811, 903, 1546, 1546, 700, 744, 2099, 978, 574, 724, 1038, 92, 1307, 909, 1323, 513, 1472, 1307, 359, 230, 135, 106, 744, 2165, 1920, 1546, 798, 170, 744, 1639, 744, 931, 1034, 2099, 1034, 1478, 2088, 1307, 744, 1411, 1077, 135, 744, 1411, 2099, 744, 1956, 574, 451, 1811, 1307, 201, 860, 268, 127, 2099, 171, 649, 2099, 170, 744, 574, 903, 744, 135, 2097, 860, 744, 744, 1308, 621, 1773, 170, 1542, 909, 1034, 744, 744, 1811, 983, 189, 840, 170, 441, 2046, 2099, 744, 700, 744, 1392, 170, 135, 1308, 1376, 2046, 135, 1652, 744, 170, 2099, 79, 249, 89, 441, 909, 847, 744, 744, 1308, 574, 79, 2214, 744,

In [48]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

54
0.2634146341463415
